In [7]:
import google.colab.drive
google.colab.drive.mount('/content/drive')

import pandas as pd
from datetime import datetime
original_df = pd.read_csv("/content/drive/MyDrive/Updated_LiveLab.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
print(len(original_df))
df = original_df[:len(original_df) // 20]
print(len(df))
df.head(5)

1650346
82517


,Anon Student Id,Session Id,Time,KC_Model(MATHia),Level (Section),Problem Name,Step Name,Attempt At Step,Help Level,Selection,...,CF (Skill New p-Known),CF (Module),CF (Workspace Encounter),CF (Workspace Variant),CF (Workspace Progress Status),CF (Skill Opportunity),cf_school_id,cf_class_id,"Selection <- ifelse(Action == ""Done Button"", ""Done Button"", NA)",helpedTransaction
0,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:26:50,match _dep expression with description.,analyzing_models_1step,analyzing_models_1step-029,gn-classify-item-4,1,0,NaN,...,0.887118,determining_unknown_quantities_mb_c1,0,empty-variant,GRADUATED,23.0,sch10,c8,NaN,False
1,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:26:53,match _dep expression with description.,analyzing_models_1step,analyzing_models_1step-029,gn-classify-item-2,1,0,NaN,...,0.970117,determining_unknown_quantities_mb_c1,0,empty-variant,GRADUATED,24.0,sch10,c8,NaN,False
2,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:27:01,match _indep expression with description.,analyzing_models_1step,analyzing_models_1step-029,gn-classify-item-1,1,0,NaN,...,0.812568,determining_unknown_quantities_mb_c1,0,empty-variant,GRADUATED,12.0,sch10,c8,NaN,False
3,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:27:07,NaN,analyzing_models_1step,analyzing_models_1step-029,gn-classify-item-1,2,0,NaN,...,NaN,determining_unknown_quantities_mb_c1,0,empty-variant,GRADUATED,NaN,sch10,c8,NaN,False
4,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:27:09,interpret scenario with words,analyzing_models_1step,analyzing_models_1step-029,gn-classify-item-3,1,0,NaN,...,0.999998,determining_unknown_quantities_mb_c1,0,empty-variant,GRADUATED,7.0,sch10,c8,NaN,False


In [60]:
selected_columns_for_unique_pairs = ['Anon Student Id', 'Session Id', 'Problem Name']
df_unique_pairs = df[selected_columns_for_unique_pairs]
df_unique_pairs = df_unique_pairs.drop_duplicates()
len(df_unique_pairs)

4278

In [75]:
selected_columns = ['Anon Student Id', 'Session Id', 'Time', 'Problem Name', 'Step Name', 'Help Level', 'Outcome', 'CF (Skill New p-Known)', 'CF (Skill Previous p-Known)', 'Attempt At Step', 'helpedTransaction']
df = original_df[selected_columns]
print('Number of helped transactions: ', df['helpedTransaction'].value_counts())
df.head()

df['Help Level'].value_counts()

Number of helped transactions:  helpedTransaction
False    1649713
True         633
Name: count, dtype: int64


,count
Help Level,
0,1356910
2,109169
1,103972
3,74890
4,5405


# Calculations

In [61]:
def is_deliberate_function(last_action, last_transaction, this_transaction):
    # Ensure Timestamp objects are converted to datetime
    time_format = "%Y-%m-%d %H:%M:%S"
    last_time = last_transaction['Time']
    this_time = this_transaction['Time']
    last_time = last_time.to_pydatetime() if isinstance(last_time, pd.Timestamp) else datetime.strptime(last_time, time_format)
    this_time = this_time.to_pydatetime() if isinstance(this_time, pd.Timestamp) else datetime.strptime(this_time, time_format)

    seconds_since_last_action = int((this_time - last_time).total_seconds())

    if last_action == 'error':
        return seconds_since_last_action > config['error_threshold']
    elif last_action in ['initial_hint', 'hint_level_change']:  # Fix logical mistake
        return seconds_since_last_action > config['hint_threshold']
    return seconds_since_last_action > config['new_step_threshold']

    # seconds_since_last_action = int((datetime.strptime(this_transaction['Time'], "%Y-%m-%d %H:%M:%S") - datetime.strptime(last_transaction['Time'], "%Y-%m-%d %H:%M:%S")).total_seconds())
    # if last_action == 'error': return seconds_since_last_action > config['error_threshold']
    # elif last_action == 'initial_hint' or 'hint_level_change': return seconds_since_last_action > config['hint_threshold'] # improvement: from calculated hint_threshold to pre-defined hint threshold
    # return seconds_since_last_action > config['new_step_threshold']

def seen_all_hint_levels_function(this_transaction):
    return True if this_transaction['Help Level'] == 4 else False

def sense_of_what_to_do_function(this_transaction):

    # Jocelyn: need the [p_know] for all skills in rawSkills to be above the senseOfWhatToDoThreshold
    return this_transaction['CF (Skill New p-Known)'] > config['sense_of_what_to_do_threshold']

def is_low_skill_step_some_function(this_transaction):
    return this_transaction['CF (Skill New p-Known)'] <= config['familiarity_threshold']

def is_familiar_function(this_transaction):
    return this_transaction['CF (Skill New p-Known)'] > config['familiarity_threshold']

In [62]:
def evaluate_action(last_transaction, this_transaction):
    last_action = last_transaction['Outcome'].lower() # hint or others
    is_correct = (this_transaction['Outcome'].lower() == "ok")
    last_action_is_error, last_action_is_hint, last_action_unclear_fix = (last_action == "error"), (last_action == 'initial_hint' or 'hint_level_change'), (last_action == 'jit')

    seen_all_hint_levels = seen_all_hint_levels_function(this_transaction)
    is_deliberate = is_deliberate_function(last_action, last_transaction, this_transaction)

    # BKT based parameters
    sense_of_what_to_do = sense_of_what_to_do_function(this_transaction)
    is_low_skill_step_some = is_low_skill_step_some_function(this_transaction)
    is_familiar = is_familiar_function(this_transaction)

    if this_transaction['Outcome'].lower() == "hint":
        print("isHint")
        if is_deliberate:
            print("isDeliberate")
            if is_low_skill_step_some and not last_action_is_error:
                return "not acceptable/asked hint on low skill step"
            else:
                if not seen_all_hint_levels and (not is_familiar or (last_action_is_error and last_action_unclear_fix) or last_action_is_hint):
                    return "preferred/ask hint"
                elif is_familiar and not sense_of_what_to_do or last_action_is_hint:
                    return "acceptable/ask hint"
                else:
                    return "hint abuse"
        else:
            print("not deliberate")
            return "hint abuse"

    else:
        if is_low_skill_step_some and not last_action_is_error:
            return "preferred/try step on low skill step"
        else:
            if is_deliberate:
                if (is_familiar and not (last_action_is_error and last_action_unclear_fix)) or (
                        last_action_is_hint):
                    return "preferred/try step"
                elif seen_all_hint_levels and not (last_action_is_error and last_action_unclear_fix):
                    return "preferred/try step"
                elif is_correct:
                    return "acceptable/try step"
                elif seen_all_hint_levels:
                    if last_action_is_error and last_action_unclear_fix:
                        return "ask teacher for help"
                else:
                    return "hint avoidance"
            else:
                return "not deliberate"

In [63]:
def is_not_deliberate(help_model_output): return help_model_output == "not deliberate"
def is_gaming(help_model_output): return False
def is_abusing_hints(help_model_output): return help_model_output == "hint abuse"

# Main function

In [64]:
config = {}
config['error_threshold'] = 2
config['new_step_threshold'] = 1
config['sense_of_what_to_do_threshold'] = 0.6
config['familiarity_threshold'] = 0.4
config['window_size'] = 10
config['hint_threshold'] = 6
config['system_misuse_threshold'] = 8 #
config['students_doing_well_threshold'] = 9 # 9 correct in 10 window size
config['idle_threshold'] = 5 # idle for >= 5 min
config['struggle_mastery_threshold'] = 0.8 # mastery threshold
config['struggle_num_attempts_threshold'] = 3 # attempt >= 3 times

In [66]:
def calculate_status_system_misuse(df, threshold):

    status, window, window_size = [], [], 10
    for i in range(len(df)):
        outcome = df['Outcome'].iloc[i]  # mostly OK, ERROR, INITIAL_HINT, JIT, HINT_LEVEL_CHANGE
        model_output = evaluate_action(df.iloc[i - 1], df.iloc[i])
        # Add to attempt window based on is gaming, is abusing hints, and is not deliberate
        attemptCorrect = 1 if is_abusing_hints(model_output) or is_not_deliberate(model_output) else 0
        window.append(attemptCorrect)
        # Check length fixed
        if len(window) == window_size: window.pop(0)
        # System_misuse window detection logic
        status.append(True) if sum(window) >= threshold else status.append(False)
    return status

In [67]:
def calculate_status_students_doing_well(df, threshold):

    status, window, window_size = [], [], 10
    for i in range(len(df)):
        # Add to attempt window based on action correct
        attemptCorrect = 1 if df['Outcome'].iloc[i].lower() == "ok" else 0
        window.append(attemptCorrect)
        # Check length fixed
        if len(window) == window_size: window.pop(0)
        # Students_doing_well detection logic
        if sum(window) >= threshold:
          status.append(True)
        else: status.append(False)
    return status

In [68]:
def calculate_status_idle(df, threshold=5):

    # Convert 'Time' column to datetime (avoiding SettingWithCopyWarning)
    df = df.copy()
    df['Time'] = pd.to_datetime(df['Time'], format='mixed', errors='coerce')

    # Sort by Session and Time
    df.sort_values(by=['Session Id', 'Time'], inplace=True)

    # Compute time difference in minutes, fill NaN with 0
    df['Time_Diff_Minutes'] = df.groupby('Session Id')['Time'].diff().dt.total_seconds().div(60).fillna(0)

    # Compute status using vectorized operations
    df['Idle'] = df['Time_Diff_Minutes'] > threshold

    return df['Idle'].tolist()  # Convert Series to list if needed


In [69]:
BKTparams = {
    "p_transit": 0.2,
    "p_slip": 0.1,
    "p_guess": 0.2,
    "p_know": 0.25  # Initial knowledge probability
}

def calculate_status_struggle(df, window_size=10, threshold=3, BKTparams=BKTparams):

    status, window = [], []

    # Initialize BKT tracking for struggle detection
    attempt_window = [0] * window_size  # Moving window for attempts
    onboardSkills = {}  # Store p_know for each skill

    for i in range(len(df)):
        outcome = df['Outcome'].iloc[i] if "Outcome" in df.columns else None  # Extract action outcome

        prev_p_know = df['CF (Skill Previous p-Known)'].iloc[i]
        new_p_know = df['CF (Skill New p-Known)'].iloc[i]
        attempts = df['Attempt At Step'].iloc[i]
        skill_name = df['Problem Name'].iloc[i]

        # Initialize skill knowledge tracking
        if skill_name not in onboardSkills:
            onboardSkills[skill_name] = BKTparams["p_know"]

        # Update p_know using Bayesian Knowledge Tracing (BKT)
        p_know_tminus1 = onboardSkills[skill_name]
        p_slip = BKTparams["p_slip"]
        p_guess = BKTparams["p_guess"]
        p_transit = BKTparams["p_transit"]

        if outcome.lower() == "ok":
            p_know_given_obs = (p_know_tminus1 * (1 - p_slip)) / ((p_know_tminus1 * (1 - p_slip)) + ((1 - p_know_tminus1) * p_guess))
        else:
            p_know_given_obs = (p_know_tminus1 * p_slip) / ((p_know_tminus1 * p_slip) + ((1 - p_know_tminus1) * (1 - p_guess)))

        onboardSkills[skill_name] = p_know_given_obs + (1 - p_know_given_obs) * p_transit

        # Add attempt to window
        if outcome.lower() == "ok":
          attemptCorrect = 1
        else: attemptCorrect = 0
        attempt_window.append(attemptCorrect)
        if len(attempt_window) > window_size:
            attempt_window.pop(0)

        # Struggle detection logic
        mastery_threshold = config['struggle_mastery_threshold']
        struggle_detected = (new_p_know < mastery_threshold or prev_p_know < mastery_threshold) and attempts >= threshold
        status.append(struggle_detected)



    return status

# Generate detectors (toy example)

In [ ]:
# Try it for one session-student-problem pair

student_id, session_id, problem_name = df_unique_pairs.iloc[0]
calculation_df = df[(df['Anon Student Id'] == student_id)]
calculation_df = calculation_df[calculation_df['Session Id'] == session_id]
calculation_df = calculation_df[calculation_df['Problem Name'] == problem_name]

In [ ]:
import numpy as np
system_misuse_calculation_global = [-1 for i in range(len(df))]

system_misuse_calculation_regional = calculate_status(calculation_df, config['window_size'], config['system_misuse_threshold'], "system_misuse")
corresponding_global_indices = list(set(np.where(df['Problem Name'] == problem_name)[0]) & set(np.where(df['Anon Student Id'] == student_id)[0]) & set(np.where(df['Session Id'] == session_id)[0]))
regional_index = 0
for global_index in corresponding_global_indices:
    system_misuse_calculation_global[global_index] = system_misuse_calculation_regional[regional_index]
    regional_index += 1
system_misuse_calculation_global

df['system_misuse_status'] = system_misuse_calculation_global
df

<ipython-input-97-fd8c1f9acc9f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['system_misuse_status'] = system_misuse_calculation_global


,Anon Student Id,Session Id,Time,Problem Name,Step Name,Help Level,Outcome,CF (Skill New p-Known),helpedTransaction,system_misuse_status
0,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:26:50,analyzing_models_1step-029,gn-classify-item-4,0,OK,0.887118,False,-1
1,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:26:53,analyzing_models_1step-029,gn-classify-item-2,0,OK,0.970117,False,-1
2,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:27:01,analyzing_models_1step-029,gn-classify-item-1,0,JIT,0.812568,False,-1
3,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:27:07,analyzing_models_1step-029,gn-classify-item-1,0,OK,NaN,False,-1
4,s141,qb8uqykv9llqylywx-ilzwqrqn0:1679667026583,2023-03-24 14:27:09,analyzing_models_1step-029,gn-classify-item-3,0,OK,0.999998,False,-1
...,...,...,...,...,...,...,...,...,...,...
1650341,s341,fzrja78lda04vk4tjj5w9wv22mm:1682432727221,2023-04-25 15:03:00,volume_surface_area_cylinder_sa-096,GN-CYLINDER-3D-SA,0,ERROR,0.666916,False,-1
1650342,s341,fzrja78lda04vk4tjj5w9wv22mm:1682432727221,2023-04-25 15:03:07,volume_surface_area_cylinder_sa-096,GN-CYLINDER-3D-SA,0,ERROR,NaN,False,-1
1650343,s341,fzrja78lda04vk4tjj5w9wv22mm:1682432727221,2023-04-25 15:03:15,volume_surface_area_cylinder_sa-096,NaN,0,ERROR,NaN,False,-1
1650344,s341,fzrja78lda04vk4tjj5w9wv22mm:1682432727221,2023-04-25 15:04:21,volume_surface_area_cylinder_sa-096,GN-CYLINDER-3D-SA,0,OK,NaN,False,-1


# Generate Detector

In [71]:
import numpy as np
system_misuse_calculation_global = [-1 for i in range(len(df))]
doing_well_calculation_global = [-1 for i in range(len(df))]
idle_calculation_global = [-1 for i in range(len(df))]
struggle_calculation_global = [-1 for i in range(len(df))]

df_unique_pairs_length_sampling = int(len(df_unique_pairs) * 1)
print(df_unique_pairs_length_sampling)

import time

start_time = time.perf_counter()

for index_selecting in range(df_unique_pairs_length_sampling):
  if index_selecting % 100 == 0:
    print(index_selecting)

  student_id, session_id, problem_name = df_unique_pairs.iloc[index_selecting]
  calculation_df = df[(df['Anon Student Id'] == student_id)]
  calculation_df = calculation_df[calculation_df['Session Id'] == session_id]
  calculation_df = calculation_df[calculation_df['Problem Name'] == problem_name]

  system_misuse_calculation_regional = calculate_status_system_misuse(calculation_df, config['system_misuse_threshold'])
  doing_well_calculation_regional = calculate_status_students_doing_well(calculation_df, config['students_doing_well_threshold'])
  idle_calculation_regional = calculate_status_idle(calculation_df)
  struggle_calculation_regional = calculate_status_struggle(calculation_df)

  corresponding_global_indices = set(np.where(df['Problem Name'] == problem_name)[0]) & set(np.where(df['Anon Student Id'] == student_id)[0]) & set(np.where(df['Session Id'] == session_id)[0])

  regional_index = 0
  for global_index in corresponding_global_indices:
      system_misuse_calculation_global[global_index] = system_misuse_calculation_regional[regional_index]
      doing_well_calculation_global[global_index] = doing_well_calculation_regional[regional_index]
      idle_calculation_global[global_index] = idle_calculation_regional[regional_index]
      struggle_calculation_global[global_index] = struggle_calculation_regional[regional_index]

      regional_index += 1

df['System Misuse'] = system_misuse_calculation_global
df['Student Doing Well'] = doing_well_calculation_global
df['Idle'] = idle_calculation_global
df['Struggle'] = struggle_calculation_global

df.head(5)

df = df.rename(columns=str.lower)

output_file_path = 'misuse_doingwell_struggle_idle_5_percentage_new_threshold.csv'
df.to_csv(output_file_path, index=False)
output_file_path  # Return the path to the new file

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

4278
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200


<ipython-input-71-474da5ff6867>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['System Misuse'] = system_misuse_calculation_global
<ipython-input-71-474da5ff6867>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Student Doing Well'] = doing_well_calculation_global
<ipython-input-71-474da5ff6867>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

Elapsed time: 194.4449 seconds


In [72]:
print(output_file_path)
df2 = pd.read_csv(output_file_path)
print(df['struggle'].value_counts())
print(df2['idle'].value_counts())
print(df2['student doing well'].value_counts())
print(df2['system misuse'].value_counts())

misuse_doingwell_struggle_idle_5_percentage_new_threshold.csv
struggle
False    82499
True        18
Name: count, dtype: int64
idle
False    82330
True       187
Name: count, dtype: int64
student doing well
False    73274
True      9243
Name: count, dtype: int64
system misuse
False    78585
True      3932
Name: count, dtype: int64
